### Vision Transformer

In [22]:
# 패키지 수입

import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor
from time import time

In [23]:
# 하이퍼 파라미터 지정

MY_SHAPE = (1, 28, 28) # 입력 이미지 형태 (MNIST: 흑백 28x28)
MY_EPOCH = 5 # 학습 반복 횟수 (성능 향상 위해 5 이상 권장)
MY_BATCH = 128 # 배치 크기 (GPU 메모리에 따라 조절 가능)
MY_LEARNING = 0.005 # 학습률 (기본 Adam 기준, 너무 크면 발산 위험)

MY_PATCH = 7 # 한 변을 나누는 패치 수 (7x7 → 총 49개 패치)
MY_ENCODER = 2 # 인코더 블록 수 (Transformer encoder layer 수)
MY_HIDDEN = 8 # 패치 임베딩 차원 수 (작으면 연산량 작아져 표현력 제한됨)
MY_HEAD = 2 # 멀티헤드 어텐션에서의 헤드 수 (각 head당 8/2=4차원)
MY_MLP = 3 # MLP 확장 비율 (hidden → hidden×3 → hidden)
MY_CLASS = 10 # 클래스 수 (MNIST: 숫자 0~9)

# GPU 사용 가능하면 GPU로 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 장치:", DEVICE)

사용 장치: cuda


In [24]:
# 이미지를 패치로 나누고 16차원 벡터로 변형
def patchify(images, n_patches): # batch = 128
    n, c, h, w = images.shape
    patch_size = h // n_patches # 28 // 7 = 4
    
    # 각 패치는 4x4=16 픽셀 -> flatten 후 16차원 벡터
    # 총 패치 수 : 7x7=49 -> 출력 shape : [128, 49, 16]
    patches = torch.zeros(n, n_patches**2, h*w*c//n_patches**2, device=images.device)
    
    for idx, image in enumerate(images):
        for i in range(n_patches): # 세로 방향 7개
            for j in range(n_patches): # 가로 방향 7개
                patch = image[
                    :,
                    i*patch_size : (i+1)*patch_size,
                    j*patch_size : (j+1)*patch_size,
                ] # patch shape : [1,4,4]
                patches[idx, i*n_patches+j] = patch.flatten() # 16차원 벡터로 변형, [16]
    return patches

In [25]:
# multi-head attention 클래스 정의
# n_hidden : 임베딩 차원 수, 8
# n_heads : 머리 수 , 2

class MyMSA(nn.Module):
    def __init__(self, n_hidden, n_heads):
        super(MyMSA, self).__init__()
        self.n_hidden = n_hidden
        self.n_heads = n_heads
        
        # 각 head 2개가 처리할 차원, 4
        d_head = int(n_hidden / n_heads) # 8/2=4
        self.d_head = d_head
        
        # Q, K, V 행렬 계산
        self.q_mappings = nn.ModuleList(
            [nn.Linear(d_head, d_head) for _ in range(n_heads)]
        )
        self.k_mappings = nn.ModuleList(
            [nn.Linear(d_head, d_head) for _ in range(n_heads)]
        )
        self.v_mappings = nn.ModuleList(
            [nn.Linear(d_head, d_head) for _ in range(n_heads)]
        )
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, images):
        # 입력 데이터 모양 : [128, 50, 8]
        # 출력 데이터 모양 : [128, 50, 8]
        
        # 128개 attention 결과 저장
        result = []
        for sequence in images:
            # 각 이미지 당 50개 패티의 계산 결과
            seq_result = []
            
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]
                
                # 입력 데이터 모양 : [50, 4]
                seq = sequence[:, head*self.d_head : (head+1)*self.d_head]
                
                # self attention 계산
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)
                
                # print(f'Q의 크기 {len(q[0])}')
                attention = self.softmax(q @ k.T / (self.d_head**0.5))
                attention = attention @ v
                #  #print('attention 크기', attention.shape)
                seq_result.append(attention)
                
            # hstack으로 두개의 head 결과 통합
            # [50, 4] + [50, 4] = [50, 8]
            merge = torch.hstack(seq_result)
            # print(f'통합 결과 {merge.shape}')
            
            # 현재 이미지 종료
            result.append(merge)
                
            # print(f'최종 배치 처리 결과 {len(result)}')
            # 결과를 텐서로 전환
            final = [torch.unsqueeze(r, dim=0) for r in result]
            final = torch.cat(final)
            # print(f'attention 결과 데이터 모양 {final.shape}')
            
            return final
        
        
# 테스트용 코드
data = torch.randn(MY_BATCH, 1, 28, 28)
y = patchify(data, MY_PATCH)
temp = MyMSA(MY_HIDDEN, MY_HEAD)
temp(y)

tensor([[[-1.8921e-01, -2.9326e-01, -1.7874e-01,  3.0636e-01,  3.3474e-01,
           4.6896e-01,  7.6117e-02,  3.3755e-01],
         [-2.9163e-01, -2.1169e-01, -2.0923e-01,  4.3220e-01,  2.7449e-01,
           5.5693e-01, -1.6262e-01,  1.5204e-01],
         [-9.0328e-02, -3.7413e-01, -1.7061e-01,  3.4751e-01,  2.6893e-01,
           5.2852e-01, -3.4628e-04,  2.7697e-01],
         [-2.1937e-01, -3.0520e-01, -1.9674e-01,  3.4087e-01,  1.7090e-01,
           6.7290e-01, -1.8733e-01,  1.3153e-01],
         [-1.6656e-01, -3.4484e-01, -1.8677e-01,  3.2935e-01,  3.5773e-01,
           3.9760e-01, -1.0853e-01,  1.9246e-01],
         [-2.7606e-01, -1.9349e-01, -1.9555e-01,  4.0996e-01,  3.0748e-01,
           4.4864e-01,  1.1297e-01,  3.6326e-01],
         [-3.3424e-01, -1.3318e-01, -1.9835e-01,  3.9397e-01,  2.4094e-01,
           5.3476e-01, -1.6465e-01,  1.4677e-01],
         [-3.2252e-01, -1.9736e-01, -1.9777e-01,  3.0896e-01,  1.6838e-01,
           6.5816e-01, -1.7565e-01,  1.3944e-01],


In [26]:
# ViT 인코더 구현
# n_hidden : 임베딩 차원 수 , 8
# n_heads : 머리 수 , 2

class MyEncoder(nn.Module):
    def __init__(self, n_hidden, n_heads):
        super(MyEncoder, self).__init__()
        
        # 패치 임베딩 차원
        self.n_hidden = n_hidden
        
        # 멀티헤드 어텐션 수
        self.n_heads = n_heads
        
        # 첫번째 layer normalization 층
        self.norm1 = nn.LayerNorm(n_hidden)
        
        # multi-head attention layer
        self.msa = MyMSA(n_hidden, n_heads)
        
        # 두번째 layer normalization layer
        self.norm2 = nn.LayerNorm(n_hidden)
        # 최종 multi-layer perceptron layer
        self.mlp = nn.Sequential(
        nn.Linear(n_hidden, MY_MLP * n_hidden),
        nn.GELU(), #가우시안 함수가 음수영역에 적용된 형태의 RELU 변형
        nn.Linear(MY_MLP * n_hidden, n_hidden)
        )
        
    # ViT 인코더 구현
    def forward(self, x):
        out = x + self.msa(self.norm1(x))
        out = self.norm2(out) + self.mlp(self.norm2(out))
        return out

In [27]:
# 6번 블록
class MyVIT(nn.Module):
    def __init__(self, n_patches, n_encoder, n_hidden, n_heads, n_class, image_shape=(1, 28, 28)):
        super(MyVIT, self).__init__()
        self.n_patches = n_patches # 7 patches
        self.n_encoder = n_encoder # 2 enconders
        self.n_heads = n_heads # 2 heads
        self.n_hidden = n_hidden # 8 dimension per 1 patch
        
        # 한 패치의 화소 수 = (패치 한 변의 길이)^2
        self.input_d = (image_shape[1] // n_patches) ** 2
        print("패치 화소 수 :", self.input_d)
        
        # 입력 차원(input_d) → 임베딩 차원(n_hidden)으로 변환
        self.linear_mapper = nn.Linear(self.input_d, n_hidden)

        # 학습 가능한 CLS 토큰 추가: [1, 1, n_hidden]
        self.class_token = nn.Parameter(torch.rand(1, 1, n_hidden)) #이미지 하나 당 [1, 1, 8] cls토큰
        print("CLS 토큰 모양 :", self.class_token.shape)
        
        # 포지셔널 인코딩: [1, 패치수+1, n_hidden]
        self.pos_embedding = nn.Parameter(torch.randn(1, n_patches**2 + 1, n_hidden))

        # 인코더 블록 여러 개 쌓기
        self.encoder = nn.Sequential(*[
        MyEncoder(n_hidden, n_heads) for _ in range(n_encoder)
        ])
        
        # 최종 분류기
        self.mlp_head = nn.Linear(n_hidden, n_class)

    def forward(self, images):
        B = images.shape[0]
        
        # 이미지를 패치 시퀀스로 변환 (patchify 함수 선행 필요)
        x = patchify(images, self.n_patches) # [B, num_patches, patch_dim]
        x = self.linear_mapper(x) # [B, num_patches, n_hidden]
        
        # CLS 토큰 복제 및 붙이기
        cls_tokens = self.class_token.expand(B, -1, -1) # [128, 1, 8] 배치128사이즈로 확장
        x = torch.cat((cls_tokens, x), dim=1) # [B, num_patches+1, n_hidden]
        
        # 포지셔널 인코딩 추가,
        x = x + self.pos_embedding # [B, 50, n_hidden]
        
        # 인코더 통과
        x = self.encoder(x) # [B, 50, n_hidden]
        
        # CLS 토큰만 추출 → 최종 분류
        cls_out = x[:, 0] # [B, n_hidden]
        logits = self.mlp_head(cls_out) # [B, n_class]
        
        return logits

In [ ]:
# 7번 블록: 학습 루프 + 테스트 평가 + 정확도 그래프
train_dataset = MNIST(root="./", train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root="./", train=False, download=True, transform=ToTensor())
train_loader = DataLoader(train_dataset, batch_size=MY_BATCH, shuffle=True) # batch_size = 128
test_loader = DataLoader(test_dataset, batch_size=MY_BATCH)

model = MyVIT(
    n_patches=MY_PATCH, # 7
    n_encoder=MY_ENCODER, # 2
    n_hidden=MY_HIDDEN, # 8
    n_heads=MY_HEAD, # 2
    n_class=MY_CLASS, # 10
    image_shape=MY_SHAPE # (1, 28, 28)
).to(DEVICE)

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=MY_LEARNING) # 학습률: 0.005

# 학습 기록용 리스트
train_accuracies = []
model.train()
start = time()

for epoch in range(MY_EPOCH): # 총 5 에폭
    epoch_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader): #train data에서 index, images, labels가 셋으로 제공
        images, labels = images.to(DEVICE), labels.to(DEVICE) # GPU로 전송
        outputs = model(images) # 출력 shape: [128, 10]
        loss = criterion(outputs, labels)
       
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
        epoch_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
    accuracy = 100 * correct / total
    train_accuracies.append(accuracy)
    print(f"[Epoch {epoch+1}] Loss: {epoch_loss:.4f} | Accuracy: {accuracy:.2f}%")

print("총 학습 시간:", round(time() - start, 2), "초")

# 정확도 그래프 출력
plt.plot(range(1, MY_EPOCH + 1), train_accuracies, marker='o')
plt.title("Training Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.grid(True)
plt.show()

# 테스트 평가
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
print(f"[Test Accuracy] {100 * correct / total:.2f}%")

패치 화소 수 : 16
CLS 토큰 모양 : torch.Size([1, 1, 8])
[Epoch 1] Loss: 1081.3224 | Accuracy: 10.50%
[Epoch 2] Loss: 1079.9939 | Accuracy: 10.76%
[Epoch 3] Loss: 1079.8567 | Accuracy: 10.86%
[Epoch 4] Loss: 1079.7857 | Accuracy: 10.98%
